# 一、文本相似度分析（基于jieba和gensim）

## 1.读取两篇文档（自己设计）：
文档1.txt
文档2.txt

In [13]:
with open('文档1.txt', 'r',encoding="utf-8") as fr: 
     text1= fr.read()
with open('文档2.txt', 'r',encoding="utf-8") as fr: 
      text2= fr.read()
print(text1)
print(text2)

我站在桥上看风景

看风景的人在桥上看我


## 2.对要计算的多篇文档进行分词

In [14]:
import jieba
texts=[text1, text2]
texts=[jieba.lcut(text) for text in texts]
print(texts)

[['我', '站', '在', '桥', '上', '看', '风景', '\n'], ['看', '风景', '的', '人', '在', '桥', '上', '看', '我']]


## 3.对文档进行整理成指定格式，方便后续进行计算

In [15]:
from gensim import corpora,models,similarities
import jieba
dictionary=corpora.Dictionary(texts)
print('词典（字典）：',dictionary.token2id)


词典（字典）： {'\n': 0, '上': 1, '在': 2, '我': 3, '桥': 4, '看': 5, '站': 6, '风景': 7, '人': 8, '的': 9}


## 4.计算出词语的频率


In [16]:
from collections import Counter
data=jieba.lcut(text1)+jieba.lcut(text2)
data=dict(Counter(data))
print(data)

{'我': 2, '站': 1, '在': 2, '桥': 2, '上': 2, '看': 3, '风景': 2, '\n': 1, '的': 1, '人': 1}


## 5.【可选】对频率低的词语进行过滤


## 6.通过语料库建立词典

In [17]:
from gensim import corpora,models,similarities
import jieba
dictionary=corpora.Dictionary(texts)
print('词典（字典）：',dictionary)
corpus=[dictionary.doc2bow(text) for text in texts]
print('语料库：',corpus)

词典（字典）： Dictionary<10 unique tokens: ['\n', '上', '在', '我', '桥']...>
语料库： [[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (7, 1), (8, 1), (9, 1)]]


## 7.加载要对比的文档


In [18]:
text3='站在桥上'

## 8.将要对比的文档通过doc2bow转化为稀疏向量

In [19]:
new_vec=dictionary.doc2bow(jieba.lcut(text3))
print(new_vec)

[(1, 1), (2, 1), (4, 1), (6, 1)]


## 9.对稀疏向量进行进一步处理，得到新语料库


In [20]:
corpus=[dictionary.doc2bow(text) for text in texts]
print('语料库：',corpus)

语料库： [[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (7, 1), (8, 1), (9, 1)]]


## 10.将新语料库通过tfidfmodel进行处理，得到tfidf


In [21]:
tfidf=models.TfidfModel(corpus)
tfidf

## 11.通过token2id得到特征数


In [22]:
num_features=len(dictionary.token2id)
num_features

10

## 12.稀疏矩阵相似度，从而建立索引


In [23]:
index=similarities.SparseMatrixSimilarity(tfidf[corpus],num_features)
print(index.index)
print('\nTF-IDF模型的稀疏向量集：')
for i in tfidf[corpus]:
    print(i)
print('\nTF-IDF模型的text3稀疏向量:')
print(tfidf[new_vec])
sim=index[tfidf[corpus]]
print(sim)

  (0, 0)	0.70710677
  (0, 6)	0.70710677
  (1, 8)	0.70710677
  (1, 9)	0.70710677

TF-IDF模型的稀疏向量集：
[(0, 0.7071067811865475), (6, 0.7071067811865475)]
[(8, 0.7071067811865475), (9, 0.7071067811865475)]

TF-IDF模型的text3稀疏向量:
[(6, 1.0)]
[[0.99999994 0.        ]
 [0.         0.99999994]]


## 13.得到最终相似度结果

In [24]:
print('\n相似度计算：')
sim=index[tfidf[new_vec]]

for i in range(len(sim)):
    print('第',i+1,'句话的相似度为:',sim[i])


相似度计算：
第 1 句话的相似度为: 0.70710677
第 2 句话的相似度为: 0.0


# 二、文本相似度分析（基于sklearn）

In [25]:
from collections import Counter
import numpy as np
import jieba
#from sklearn.metrics.pairwise import cosine_similarity

words1_dict=dict(Counter(jieba.lcut(text1)))
words2_dict=dict(Counter(jieba.lcut(text2)))
print(words1_dict)
print(words2_dict)
dic1 = sorted(words1_dict.items(), key=lambda x: x[1], reverse=True)
dic2 = sorted(words2_dict.items(), key=lambda x: x[1], reverse=True)
print(dic1)
print(dic2)
words_key = []
list(map(lambda x: words_key.append(x[0]), dic1))
list(map(lambda x: words_key.append(x[0]), filter(lambda x: x[0] not in words_key, dic2)))
print(words_key)
vect1 = []
vect2 = []
for word in words_key:
    if word in words1_dict:
        vect1.append(words1_dict[word])
    else:
         vect1.append(0)
    if word in words2_dict:
         vect2.append(words2_dict[word])
    else:
         vect2.append(0)
print(vect1)
print(vect2)
from sklearn.metrics.pairwise import cosine_similarity
user_tag_matric = np.matrix(np.array([vect1, vect2]))
user_similarity = cosine_similarity(user_tag_matric)
print(user_similarity)

{'我': 1, '站': 1, '在': 1, '桥': 1, '上': 1, '看': 1, '风景': 1, '\n': 1}
{'看': 2, '风景': 1, '的': 1, '人': 1, '在': 1, '桥': 1, '上': 1, '我': 1}
[('我', 1), ('站', 1), ('在', 1), ('桥', 1), ('上', 1), ('看', 1), ('风景', 1), ('\n', 1)]
[('看', 2), ('风景', 1), ('的', 1), ('人', 1), ('在', 1), ('桥', 1), ('上', 1), ('我', 1)]
['我', '站', '在', '桥', '上', '看', '风景', '\n', '的', '人']
[1, 1, 1, 1, 1, 1, 1, 1, 0, 0]
[1, 0, 1, 1, 1, 2, 1, 0, 1, 1]
[[1.         0.74620251]
 [0.74620251 1.        ]]
